In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import sys
# model file directory
sys.path.append('/content/drive/My Drive/pymodule')

import os
import json
import numpy as np
from tqdm import tqdm
import librosa
import librosa.display
import shutil

import matplotlib.pyplot as plt

from os import listdir
from os.path import isfile, join

from os.path import join
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dense, Dropout, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
from keras.models import Sequential
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 

from cnn8layer import Cnn8layer

Using TensorFlow backend.


In [0]:
RANDOM_STATE = 42
num_channels = 1
num_rows = 128
num_columns = 20
num_labels = 3
checkpointer = '/content/drive/My Drive/makinter_checkpoint/8layer_tri_v2_0131.hdf5'
#output_dir = '/content/drive/My Drive/makinter_checkpoint/8layer.hdf5'

In [0]:
# train test split
datapath = '/content/drive/My Drive/Kickboard_Software/real_data'

as_data = '/content/drive/My Drive/Kickboard_Software/real_data/as'
ce_data = '/content/drive/My Drive/Kickboard_Software/real_data/ce'
ur_data = '/content/drive/My Drive/Kickboard_Software/real_data/ur'

as_arr_data = '/content/drive/My Drive/Kickboard_Software/real_data/as/arr'
ce_arr_data = '/content/drive/My Drive/Kickboard_Software/real_data/ce/arr'
ur_arr_data = '/content/drive/My Drive/Kickboard_Software/real_data/ur/arr'

as_train_dir = '/content/drive/My Drive/Kickboard_Software/real_data/train/as'
ce_train_dir = '/content/drive/My Drive/Kickboard_Software/real_data/train/ce'
ur_train_dir = '/content/drive/My Drive/Kickboard_Software/real_data/train/ur'

as_test_dir = '/content/drive/My Drive/Kickboard_Software/real_data/test/as'
ce_test_dir = '/content/drive/My Drive/Kickboard_Software/real_data/test/ce'
ur_test_dir = '/content/drive/My Drive/Kickboard_Software/real_data/test/ur'

as_files = [f for f in listdir(as_data) if isfile(join(as_data,f))]
as_train, as_test = train_test_split(as_files, test_size=0.1, random_state = RANDOM_STATE)
print("as train")
print(len(as_train))
print("as test")
print(len(as_test))

for i in tqdm(range(len(as_test))):
  shutil.move(join(as_data, as_test[i]), join(as_test_dir, as_test[i])) 

ce_files = [f for f in listdir(ce_data) if isfile(join(ce_data,f))]
ce_train, ce_test = train_test_split(ce_files, test_size=0.1, random_state = RANDOM_STATE)
print("ce train")
print(len(ce_train))
print("ce test")
print(len(ce_test))

for i in tqdm(range(len(ce_test))):
  shutil.move(join(ce_data, ce_test[i]), join(ce_test_dir, ce_test[i])) 

ur_files = [f for f in listdir(ur_data) if isfile(join(ur_data,f))]
ur_train, ur_test = train_test_split(ur_files, test_size=0.1, random_state = RANDOM_STATE)
print("ur train")
print(len(ur_train))
print("ur test")
print(len(ur_test))

for i in tqdm(range(len(ur_test))):
  shutil.move(join(ur_data, ur_test[i]), join(ur_test_dir, ur_test[i])) 

  0%|          | 0/600 [00:00<?, ?it/s]

as train
5400
as test
600


  0%|          | 0/570 [00:00<?, ?it/s]

ce train
5130
ce test
570


  0%|          | 0/606 [00:00<?, ?it/s]

ur train
5454
ur test
606


100%|██████████| 606/606 [00:05<00:00, 113.36it/s]


In [0]:
# create train data
as_data = '/content/drive/My Drive/Kickboard_Software/real_data/as'
ce_data = '/content/drive/My Drive/Kickboard_Software/real_data/ce'
ur_data = '/content/drive/My Drive/Kickboard_Software/real_data/ur'

as_train = [f for f in listdir(as_data) if isfile(join(as_data,f))]
print(len(as_train))                                                    

ce_train = [f for f in listdir(ce_data) if isfile(join(ce_data,f))]
print(len(ce_train))                                                    

ur_train = [f for f in listdir(ur_data) if isfile(join(ur_data,f))]
print(len(ur_train))                                                    

# noise testing
"""
noise = np.random.normal(0,30,10000)
print(noise)
noise = np.random.normal(0,30,10000)
print(noise)
"""
"""
noise = np.random.normal(0,30,10000)
div = (np.max(noise) - np.min(noise))/2.
if div == 0:
    div = 1.

noise = noise / div
noise = noise - noise.mean(axis=0)
noise_S = librosa.feature.melspectrogram(y=noise,sr=10000)
"""
"""
S_DB = librosa.power_to_db(S, ref=np.max)
librosa.display.specshow(S_DB, sr=10000, hop_length=512, x_axis='time', y_axis='mel');
plt.colorbar(format='%+2.0f dB');
"""
"""
feat = np.load(join(as_data,as_train[0]))
S_DB = librosa.power_to_db(feat, ref=np.max)
librosa.display.specshow(S_DB, sr=10000, hop_length=512, x_axis='time', y_axis='mel');
plt.colorbar(format='%+2.0f dB');
"""
dataset_x = []
dataset_y = []

as_cnt = 0
print("loading features for asphalt")
for feat_file in tqdm(as_train):
  try:
    feat = np.load(join(as_data, feat_file))
    dataset_x.append(feat)
    dataset_y.append(1)
    as_cnt += 1
    """
    for i in range(5):
      noise = np.random.normal(0,30,10000)
      div = (np.max(noise) - np.min(noise))/2.
      if div == 0:
          div = 1.
      noise = noise / div
      noise = noise - noise.mean(axis=0)
      noise_S = librosa.feature.melspectrogram(y=noise,sr=10000)
      noise_added = feat + noise_S*0.01
      dataset_x.append(noise_added)
      dataset_y.append(1)
      as_cnt += 1
    """
    
  except:
    print("unable to load")
    pass
print("finished loading %d features for asphalt" % as_cnt)

ce_cnt = 0
print("loading features for ce")
for feat_file in tqdm(ce_train):
  try:
    feat = np.load(join(ce_data, feat_file))
    dataset_x.append(feat)
    dataset_y.append(0)
    ce_cnt += 1

    """

    for i in range(5):
      noise = np.random.normal(0,30,10000)
      div = (np.max(noise) - np.min(noise))/2.
      if div == 0:
          div = 1.
      noise = noise / div
      noise = noise - noise.mean(axis=0)
      noise_S = librosa.feature.melspectrogram(y=noise,sr=10000)
      noise_added = feat + noise_S*0.01
      dataset_x.append(noise_added)
      dataset_y.append(0)
      ce_cnt += 1
    """
  except:
    print("unable to load")
    pass
print("finished loading %d features for ce" % ce_cnt)

ur_cnt = 0
print("loading features for ur")
for feat_file in tqdm(ur_train):
  try:
    feat = np.load(join(ur_data, feat_file))
    dataset_x.append(feat)
    dataset_y.append(2)
    ur_cnt += 1

    """
    for i in range(5):
      noise = np.random.normal(0,30,10000)
      div = (np.max(noise) - np.min(noise))/2.
      if div == 0:
          div = 1.
      noise = noise / div
      noise = noise - noise.mean(axis=0)
      noise_S = librosa.feature.melspectrogram(y=noise,sr=10000)
      noise_added = feat + noise_S*0.01
      dataset_x.append(noise_added)
      dataset_y.append(2)
      ur_cnt += 1
    """
  except:
    print("unable to load")
    pass
print("finished loading %d features for ur" % ur_cnt)

x = np.array(dataset_x)
y = np.array(dataset_y)
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

_x_train, __x, _y_train, __y = train_test_split(x,yy,test_size=0.001, random_state = RANDOM_STATE)
print("len train x: ", len(_x_train))
print("len train y: ", len(_y_train))
_x_train = _x_train.reshape(_x_train.shape[0],num_rows, num_columns, num_channels)

np.save('/content/drive/My Drive/Kickboard_Software/_x_train_v2_0131',_x_train)
np.save('/content/drive/My Drive/Kickboard_Software/_y_train_v2_0131',_y_train)

5940
5970


  0%|          | 0/5940 [00:00<?, ?it/s]

5934
loading features for asphalt


  0%|          | 0/5970 [00:00<?, ?it/s]

finished loading 5940 features for asphalt
loading features for ce


  0%|          | 0/5934 [00:00<?, ?it/s]

finished loading 5970 features for ce
loading features for ur


100%|██████████| 5934/5934 [20:34<00:00,  2.36it/s]


finished loading 5934 features for ur
len train x:  17826
len train y:  17826


In [0]:
#create test data

as_test_dir = '/content/drive/My Drive/Kickboard_Software/real_data/test/as'
ce_test_dir = '/content/drive/My Drive/Kickboard_Software/real_data/test/ce'
ur_test_dir = '/content/drive/My Drive/Kickboard_Software/real_data/test/ur'

dataset_x = []
dataset_y = []

ce_files = os.listdir(ce_test_dir)
ce_cnt = 0
print("loading test features for cement")
for feat_file in tqdm(ce_files):
    try:
      feat = np.load(join(ce_test_dir, feat_file))
      dataset_x.append(feat)
      dataset_y.append(0)
      ce_cnt += 1
    except:
        print("unable to load")
        pass
print("finished loading %d features for cement test" % ce_cnt)

as_files = os.listdir(as_test_dir)
as_cnt = 0
print("loading test features for asphalt")
for feat_file in tqdm(as_files):
  try:
    feat = np.load(join(as_test_dir, feat_file))
    dataset_x.append(feat)
    dataset_y.append(1)
    as_cnt += 1
  except:
    print("unable to load")
    pass
print("finished loading %d features for asphalt test" % as_cnt)

ur_files = os.listdir(ur_test_dir)
ur_cnt = 0
print("loading features for ur test")
for feat_file in tqdm(ur_files):
  try:
    feat = np.load(join(ur_test_dir, feat_file))
    dataset_x.append(feat)
    dataset_y.append(2)
    ur_cnt += 1
  except:
    print("unable to load")
    pass
print("finished loading %d features for ur test" % ur_cnt)


x = np.array(dataset_x)
y = np.array(dataset_y)
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

__, _x_test, _, _y_test = train_test_split(x,yy,test_size=0.99, random_state = RANDOM_STATE)

print("len test x: ", len(_x_test))
print("len test y: ", len(_y_test))
_x_test = _x_test.reshape(_x_test.shape[0], num_rows, num_columns, num_channels)

np.save('/content/drive/My Drive/Kickboard_Software/_x_test',_x_test)
np.save('/content/drive/My Drive/Kickboard_Software/_y_test',_y_test)

 15%|█▍        | 83/570 [00:00<00:00, 824.79it/s]

loading test features for cement


  0%|          | 0/600 [00:00<?, ?it/s]

finished loading 570 features for cement test
loading test features for asphalt


 13%|█▎        | 76/606 [00:00<00:00, 756.87it/s]

finished loading 600 features for asphalt test
loading features for ur test


100%|██████████| 606/606 [02:11<00:00,  3.96it/s]


finished loading 606 features for ur test
len test x:  1759
len test y:  1759


In [0]:
# loading data
_x_train = np.load('/content/drive/My Drive/Kickboard_Software/_x_train_v2_0131.npy')
_x_test = np.load('/content/drive/My Drive/Kickboard_Software/_x_test.npy')
_y_train = np.load('/content/drive/My Drive/Kickboard_Software/_y_train_v2_0131.npy')
_y_test = np.load('/content/drive/My Drive/Kickboard_Software/_y_test.npy')

In [0]:
# train
print(num_rows)
print(num_columns)
model = Cnn8layer(num_rows, num_columns, num_channels, num_labels)

num_epochs = 10 
num_batch_size = 32 
model.train(x_train=_x_train, x_test=_x_test, y_train=_y_train, y_test=_y_test, num_epochs=num_epochs, num_batch_size=num_batch_size, checkpoint = checkpointer)


128
20




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 127, 19, 32)       160       
_________________________________________________________________
dropout_1 (Dropout)          (None, 127, 19, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 126, 18, 32)       4128      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 9, 32)         0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 63, 9, 32)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 62, 8, 64)         82